### Imports

In [36]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [37]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [38]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [39]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [40]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [41]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-72    8.6   83     0.69   0        Ruim     1.00      
-84    3.6   37     0.71   0        Ruim     0.99      
-89    8.8   63     0.45   0        Ruim     1.00      
-36    3.0   22     0.20   1        Bom      1.00      
-58    4.8   53     0.75   0        Ruim     1.00      
-57    1.5   22     0.37   1        Bom      0.99      
-83    7.4   89     0.37   0        Ruim     1.00      
-67    6.5   86     0.51   0        Ruim     1.00      
-50    3.6   23     0.26   1        Bom      0.98      
-68    5.2   40     0.90   0        Ruim     1.00      
-87    9.6   65     0.66   0        Ruim     1.00      
-32    3.0   23     0.21   1        Bom      1.00      
-62    4.2   87     0.92   0        Ruim     1.00      
-67    3.2   36     0.32   1        Bom      0.82      
-65    3.0   36     0.22   1        Bom      0.94      
-37    2.5   31     0.29   1        Bom      1.0

### Save

In [42]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
